# Notebook demonstrating how to access GOES fire detection alerts data from geoserver

<img src='https://earthobservatory.nasa.gov/ContentFeature/GlobalFire/Images/goes_fire.jpg'><img src='https://www.spaceflightinsider.com/wp-content/uploads/2017/02/GOES-R_Earth-Reflection-2012_rsz-1600x1060.jpg' width='260'>




**What are Geostationary Operational Environmental Satellite (GOES) alerts and how do you query alerts data?**

The <a href = "https://earthobservatory.nasa.gov/features/GlobalFire/fire_5.php">Geostationary Operational Environmental Satellites (GOES)</a> house a five-channel (one visible, four infrared) imaging radiometer designed to sense radiant and solar reflected energy from sample areas of the Earth. They are stationed in orbits that remain fixed over one spot on the equator, providing continuous coverage of one hemisphere. GOES satellites aquire images every 15–30 minutes, at up to 1 km resolution in visible light, for the detection of smoke, and 4 km resolution in thermal infrared to directly detect the heat of fires. 

<a href = "https://wifire-data.sdsc.edu/dataset/goes-fire-detections">GOES fire detection alert data</a> shown in this notebook are generated from GOES16 and GOES17 satellites and accessed from <a href = "https://sdge.sdsc.edu/geoserver">geoserver </a>using the <a href = "https://docs.geoserver.org/stable/en/user/services/wfs/basics.html">Web Feature Service (WFS)</a> specification.

**References:**
    
- WFS: https://docs.geoserver.org/latest/en/user/services/wfs/reference.html
- cql filter: https://docs.geoserver.org/stable/en/user/tutorials/cql/cql_tutorial.html

<h3>Notebook Overview:</h3>
<ol><a href='#recent_alerts'><li>Query most recent GOES alert</li></a>
    <a href='#10recent_alerts'><li>Query 10 most recent alerts</li></a>
    <a href='#timestamps'><li>Query alerts between particular timestamps t1 and t2</li></a>
    <a href='#bbox'><li>Query alerts by bounding box</li></a>
    <a href='#detections'><li>Query alerts for only high quality detections</li></a>
    <a href='#plot'><li>Plot results on map and save data as geojson and shapefile</li></a>

In [1]:
# cartopy installation recommended for map plotting visualizations
# !conda install cartopy

In [2]:
import requests, json, pprint
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd
from io import StringIO

plt.style.use('ggplot')

# Here are all the features for GOES alerts

In [3]:
#let's take a look at a query geojson response in geopandas

baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = "service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&outputFormat=application%2Fjson" 

url = baseURL + selectionParameters

#url = 'https://sdge.sdsc.edu/geoserver/SDGE/ows?service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&outputFormat=application%2Fjson'

df = gpd.read_file(url)
df.head()

,id,filename,algorithm,sector,satellite,server_name,data_time,created_time,code,frp,...,obs_bt11,bkg_bt4,bkg_bt11,solar_zenith,satellite_zenith,relative_azimuth,ecosystem,fd_x,fd_y,geometry
0,1685586,f2021351193617.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2021-12-17T19:36:17+00:00,2021-12-17T19:39:20.677000+00:00,10,41.9,...,293.836,299.390,293.496,55.786,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)
1,1685587,f2021351194117.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2021-12-17T19:41:17+00:00,2021-12-17T19:44:19.330000+00:00,10,83.9,...,294.018,299.378,293.551,55.801,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)
2,1685588,f2021351194617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2021-12-17T19:46:17+00:00,2021-12-17T19:49:20.732000+00:00,10,34.2,...,295.469,301.348,294.573,55.854,44.954,-9999,37,3671,1078,POINT (-115.12080 32.54780)
3,1685589,f2021351194617.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2021-12-17T19:46:17+00:00,2021-12-17T19:49:25.041000+00:00,10,52.8,...,293.897,299.402,293.607,55.843,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)
4,1685590,f2021351194032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2021-12-17T19:40:32+00:00,2021-12-17T19:50:11.584000+00:00,12,43.1,...,294.055,301.039,294.176,55.809,44.969,-9999,37,3672,1078,POINT (-115.09600 32.54870)


<a id='recent_alerts'><h3>Query for the most recent GOES alert</h3></a>

#### This is a typical query reponse in geojson output

In [4]:
baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = "service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&count=1&sortBy=data_time&outputFormat=application%2Fjson" 

url = baseURL + selectionParameters

r = requests.get(url) 
data_dict = r.json()
data_dict


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'ssec_wfabba_goes.fid-7c6661d7_17dcab830ac_-7b31',
   'geometry': {'type': 'Point', 'coordinates': [-115.1088, 32.5361]},
   'geometry_name': 'geom',
   'properties': {'id': 1685586,
    'filename': 'f2021351193617.firelist.6_6_001g.FDCC.GOES-16.txt',
    'algorithm': '6_6_001g',
    'sector': 'FDCC',
    'satellite': 'GOES-16',
    'server_name': None,
    'data_time': '2021-12-17T19:36:17Z',
    'created_time': '2021-12-17T19:39:20.677Z',
    'code': 10,
    'frp': 41.9,
    'fire_size': 6331,
    'fire_temp': 622.3,
    'line_num': 144,
    'element_num': 136,
    'pixel_size': 8485020,
    'obs_bt4': 305.673,
    'obs_bt11': 293.836,
    'bkg_bt4': 299.39,
    'bkg_bt11': 293.496,
    'solar_zenith': 55.786,
    'satellite_zenith': 57.302,
    'relative_azimuth': -9999,
    'ecosystem': 37,
    'fd_x': 1096,
    'fd_y': 1116}}],
 'totalFeatures': 25,
 'numberMatched': 25,
 'numberReturned': 1,
 'timeStamp': '2

<a id='10recent_alerts'><h3>Query for the 10 most recent GOES alerts</h3></a>

In [5]:
baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = "service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&count=10&sortBy=data_time&outputFormat=application%2Fjson" 

url = baseURL + selectionParameters

#view in geopandas
most_recent_10_alerts = gpd.read_file(url)
most_recent_10_alerts
#most_recent_10['data_time']

# move forward with the geojson data

#r = requests.get(url) 
#data_dict = r.json()
#data_dict

,id,filename,algorithm,sector,satellite,server_name,data_time,created_time,code,frp,...,obs_bt11,bkg_bt4,bkg_bt11,solar_zenith,satellite_zenith,relative_azimuth,ecosystem,fd_x,fd_y,geometry
0,1685586,f2021351193617.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2021-12-17T19:36:17+00:00,2021-12-17T19:39:20.677000+00:00,10,41.9,...,293.836,299.390,293.496,55.786,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)
1,1685591,f2021351194020.firelist.6_6_001g.FDCF.GOES-16.txt,6_6_001g,FDCF,GOES-16,None,2021-12-17T19:40:20+00:00,2021-12-17T19:50:34.438999+00:00,10,62.4,...,293.957,299.393,293.520,55.796,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)
2,1685590,f2021351194032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2021-12-17T19:40:32+00:00,2021-12-17T19:50:11.584000+00:00,12,43.1,...,294.055,301.039,294.176,55.809,44.969,-9999,37,3672,1078,POINT (-115.09600 32.54870)
3,1685587,f2021351194117.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2021-12-17T19:41:17+00:00,2021-12-17T19:44:19.330000+00:00,10,83.9,...,294.018,299.378,293.551,55.801,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)
4,1685589,f2021351194617.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2021-12-17T19:46:17+00:00,2021-12-17T19:49:25.041000+00:00,10,52.8,...,293.897,299.402,293.607,55.843,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)
5,1685588,f2021351194617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2021-12-17T19:46:17+00:00,2021-12-17T19:49:20.732000+00:00,10,34.2,...,295.469,301.348,294.573,55.854,44.954,-9999,37,3671,1078,POINT (-115.12080 32.54780)
6,1685593,f2021351195032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2021-12-17T19:50:32+00:00,2021-12-17T20:00:12.834000+00:00,10,34.8,...,295.439,301.489,294.675,55.905,44.954,-9999,37,3671,1078,POINT (-115.12080 32.54780)
7,1685592,f2021351195617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2021-12-17T19:56:17+00:00,2021-12-17T19:59:20.756001+00:00,12,53.7,...,295.708,302.378,294.905,56.141,45.113,-9999,51,3673,1073,POINT (-115.03480 32.66970)
8,1685594,f2021351200032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2021-12-17T20:00:32+00:00,2021-12-17T20:10:12.899000+00:00,12,45.9,...,295.708,302.487,295.018,56.234,45.113,-9999,51,3673,1073,POINT (-115.03480 32.66970)
9,1685600,f2021351202032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2021-12-17T20:20:32+00:00,2021-12-17T20:30:11.913000+00:00,12,39.3,...,294.025,300.978,294.138,56.710,44.916,-9999,37,3677,1083,POINT (-115.00790 32.43320)


<a id='timestamps'><h3>Query for GOES alerts between timestamps t1 and t2 using cql_filter</h3></a>

reference: https://docs.geoserver.org/stable/en/user/tutorials/cql/cql_tutorial.html

## let's choose a time window to query between

In [6]:
t1 = '2021-12-17T19:36:17Z'
t2 = '2021-12-17T19:46:17Z'

baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = f'service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&cql_filter=data_time%20BETWEEN%20{t1}AND%20{t2}&outputFormat=application%2Fjson'

url = baseURL + selectionParameters

#view in geopandas
time_window_query = gpd.read_file(url)
time_window_query
#most_recent_10['data_time']

# move forward with the geojson data

#r = requests.get(url) 
#data_dict = r.json()
#data_dict

,id,filename,algorithm,sector,satellite,server_name,data_time,created_time,code,frp,...,obs_bt11,bkg_bt4,bkg_bt11,solar_zenith,satellite_zenith,relative_azimuth,ecosystem,fd_x,fd_y,geometry
0,1685586,f2021351193617.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2021-12-17T19:36:17+00:00,2021-12-17T19:39:20.677000+00:00,10,41.9,...,293.836,299.390,293.496,55.786,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)
1,1685591,f2021351194020.firelist.6_6_001g.FDCF.GOES-16.txt,6_6_001g,FDCF,GOES-16,None,2021-12-17T19:40:20+00:00,2021-12-17T19:50:34.438999+00:00,10,62.4,...,293.957,299.393,293.520,55.796,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)
2,1685590,f2021351194032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2021-12-17T19:40:32+00:00,2021-12-17T19:50:11.584000+00:00,12,43.1,...,294.055,301.039,294.176,55.809,44.969,-9999,37,3672,1078,POINT (-115.09600 32.54870)
3,1685587,f2021351194117.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2021-12-17T19:41:17+00:00,2021-12-17T19:44:19.330000+00:00,10,83.9,...,294.018,299.378,293.551,55.801,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)
4,1685589,f2021351194617.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2021-12-17T19:46:17+00:00,2021-12-17T19:49:25.041000+00:00,10,52.8,...,293.897,299.402,293.607,55.843,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)
5,1685588,f2021351194617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2021-12-17T19:46:17+00:00,2021-12-17T19:49:20.732000+00:00,10,34.2,...,295.469,301.348,294.573,55.854,44.954,-9999,37,3671,1078,POINT (-115.12080 32.54780)


<a id='detections'><h3>Query for only quality detections</h3></a>

The quality of the fire detections is indicated by the 'code' feature. For example, a value of 15 and 35 is considered a low probability.  Here are the codes:

Code: 
- 10 (30) - Processed Fire Pixel (Temporally filtered)
- 11 (31) - Saturated Fire Pixel (Temporally filtered)
- 12 (32) - Cloudy Fire Pixel (Temporally filtered)
- 13 (33) - High Probability Fire Pixel (Temporally filtered)
- 14 (34) - Medium Probability Fire Pixel (Temporally filtered)
- 15 (35) - Low Probability Fire Pixel (Temporally filtered)  

**Let's query for fire detections with a Cloudy Fire Pixel (code = 12)**

In [7]:
code_value='12'

baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = f'service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&count=100&cql_filter=code={code_value}&outputFormat=application%2Fjson'

url = baseURL + selectionParameters

#view in geopandas
time_window_query = gpd.read_file(url)
time_window_query
#most_recent_10['data_time']

# move forward with the geojson data

#r = requests.get(url) 
#data_dict = r.json()
#data_dict

,id,filename,algorithm,sector,satellite,server_name,data_time,created_time,code,frp,...,obs_bt11,bkg_bt4,bkg_bt11,solar_zenith,satellite_zenith,relative_azimuth,ecosystem,fd_x,fd_y,geometry
0,1685590,f2021351194032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2021-12-17T19:40:32+00:00,2021-12-17T19:50:11.584000+00:00,12,43.1,...,294.055,301.039,294.176,55.809,44.969,-9999,37,3672,1078,POINT (-115.09600 32.54870)
1,1685592,f2021351195617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2021-12-17T19:56:17+00:00,2021-12-17T19:59:20.756001+00:00,12,53.7,...,295.708,302.378,294.905,56.141,45.113,-9999,51,3673,1073,POINT (-115.03480 32.66970)
2,1685594,f2021351200032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2021-12-17T20:00:32+00:00,2021-12-17T20:10:12.899000+00:00,12,45.9,...,295.708,302.487,295.018,56.234,45.113,-9999,51,3673,1073,POINT (-115.03480 32.66970)
3,1685595,f2021351202117.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2021-12-17T20:21:17+00:00,2021-12-17T20:24:20.657000+00:00,12,49.7,...,296.066,302.560,295.123,56.982,45.113,-9999,51,3673,1073,POINT (-115.03480 32.66970)
4,1685597,f2021351202617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2021-12-17T20:26:17+00:00,2021-12-17T20:29:19.257000+00:00,12,47.0,...,295.708,302.599,295.127,57.223,45.113,-9999,51,3673,1073,POINT (-115.03480 32.66970)
5,1685599,f2021351202032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2021-12-17T20:20:32+00:00,2021-12-17T20:30:11.889000+00:00,12,50.0,...,295.768,302.661,295.077,56.936,45.113,-9999,51,3673,1073,POINT (-115.03480 32.66970)
6,1685600,f2021351202032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2021-12-17T20:20:32+00:00,2021-12-17T20:30:11.913000+00:00,12,39.3,...,294.025,300.978,294.138,56.710,44.916,-9999,37,3677,1083,POINT (-115.00790 32.43320)
7,1685602,f2021351204617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2021-12-17T20:46:17+00:00,2021-12-17T20:49:20.032000+00:00,12,49.6,...,295.917,302.393,295.043,58.423,45.113,-9999,51,3673,1073,POINT (-115.03480 32.66970)
8,1685605,f2021351212617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2021-12-17T21:26:17+00:00,2021-12-17T21:29:18.486000+00:00,12,48.8,...,295.409,301.051,294.312,61.864,45.113,-9999,51,3673,1073,POINT (-115.03480 32.66970)
9,1685606,f2021351212020.firelist.6_6_001g.FDCF.GOES-16.txt,6_6_001g,FDCF,GOES-16,None,2021-12-17T21:20:20+00:00,2021-12-17T21:30:33.380001+00:00,12,63.5,...,292.800,296.915,292.724,61.118,57.302,-9999,37,1096,1116,POINT (-115.10880 32.53610)


<a id='bbox'><h3>Query alerts by bounding box</h3></a>

In [8]:
# code here

<a id='plot'><h3>Plot results on map and demonstrate how to save data as geojson and shapefile</h3></a>

In [9]:
#code here

#### save as shape file

In [10]:
url = 'https://sdge.sdsc.edu/geoserver/SDGE/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=SDGE%3Assec_wfabba_goes&maxFeatures=50&outputFormat=SHAPE-ZIP'

r = requests.get(url) 

with open("ssec_wfabba_goes.zip", "wb") as f:
    f.write(r.content)